In [15]:
#
# Bin packing as a LP problem:
# http://www.or.deis.unibo.it/kp/Chapter8.pdf
#
# Requisite Wiki Article:
# https://en.wikipedia.org/wiki/Bin_packing_problem
#
# PuLP Library:
# https://pythonhosted.org/PuLP/index.html
#

from pulp import LpVariable,LpProblem,lpSum
import time

#
# A list of item tuples (name, weight) -- name is meaningless except to humans.
# Weight and Size are used interchangeably here and elsewhere.
#
items = [("a", 5),
         ("b", 6),
         ("c", 7),
         ("d", 32),
         ("e", 2),
         ("f", 32),
         ("g", 5),
         ("h", 7),
         ("i", 9),
         ("k", 12),
         ("l", 11),
         ("m", 1),
         ("n", 2)]

itemCount = len(items)

# Max number of bins allowed.
maxBins = 32

# Bin Size
binCapacity = 32

In [21]:
# Indicator variable assigned 1 when the bin is used.
y = LpVariable.dicts('BinUsed', range(maxBins), 
                            lowBound = 0,
                            upBound = 1,
                            cat = pulp.LpInteger)

# An indicator variable that is assigned 1 when item is placed into binNum
possible_ItemInBin = [(itemTuple[0], binNum) for itemTuple in items
                                            for binNum in range(maxBins)]
x = LpVariable.dicts('itemInBin', possible_ItemInBin,
                            lowBound = 0,
                            upBound = 1,
                            cat = pulp.LpInteger)

# Initialize the problem
prob = LpProblem("Bin Packing Problem", LpMinimize)

# Add the objective function.
prob += lpSum([y[i] for i in range(maxBins)]), "Objective: Minimize Bins Used"

In [22]:
#
# This is the constraints section.
#

# First constraint: For every item, the sum of bins in which it appears must be 1
for j in items:
    prob += lpSum([x[(j[0], i)] for i in range(maxBins)]) == 1, ("An item can be in only 1 bin -- " + str(j[0]))

# Second constraint: For every bin, the number of items in the bin cannot exceed the bin capacity
for i in range(maxBins):
    prob += lpSum([items[j][1] * x[(items[j][0], i)] for j in range(itemCount)]) <= binCapacity*y[i], ("The sum of item sizes must be smaller than the bin -- " + str(i))

# Write the model to disk
prob.writeLP("BinPack.lp")


In [20]:
# Solve the optimization.
start_time = time.time()
prob.solve()
print("Solved in %s seconds." % (time.time() - start_time))


# Bins used
print("Bins used: " + str(sum(([y[i].value() for i in range(maxBins)]))))

# The rest of this is some unpleasent massaging to get pretty results.
bins = {}
for itemBinPair in x.keys():
    if(x[itemBinPair].value() == 1):
        itemNum = itemBinPair[0]
        binNum = itemBinPair[1]
        if binNum in bins:
            bins[binNum].append(itemNum)
        else:
            bins[binNum] = [itemNum]

for b in bins.keys():
    print(str(b) + ": " + str(bins[b]))

Solved in 0.0547881126404 seconds.
Bins used: 5.0
0: ['m', 'a', 'n', 'e', 'i', 'b']
1: ['l', 'c', 'g', 'h']
12: ['d']
13: ['f']
7: ['k']


In [24]:
print possible_ItemInBin

[('a', 0), ('a', 1), ('a', 2), ('a', 3), ('a', 4), ('a', 5), ('a', 6), ('a', 7), ('a', 8), ('a', 9), ('a', 10), ('a', 11), ('a', 12), ('a', 13), ('a', 14), ('a', 15), ('a', 16), ('a', 17), ('a', 18), ('a', 19), ('a', 20), ('a', 21), ('a', 22), ('a', 23), ('a', 24), ('a', 25), ('a', 26), ('a', 27), ('a', 28), ('a', 29), ('a', 30), ('a', 31), ('b', 0), ('b', 1), ('b', 2), ('b', 3), ('b', 4), ('b', 5), ('b', 6), ('b', 7), ('b', 8), ('b', 9), ('b', 10), ('b', 11), ('b', 12), ('b', 13), ('b', 14), ('b', 15), ('b', 16), ('b', 17), ('b', 18), ('b', 19), ('b', 20), ('b', 21), ('b', 22), ('b', 23), ('b', 24), ('b', 25), ('b', 26), ('b', 27), ('b', 28), ('b', 29), ('b', 30), ('b', 31), ('c', 0), ('c', 1), ('c', 2), ('c', 3), ('c', 4), ('c', 5), ('c', 6), ('c', 7), ('c', 8), ('c', 9), ('c', 10), ('c', 11), ('c', 12), ('c', 13), ('c', 14), ('c', 15), ('c', 16), ('c', 17), ('c', 18), ('c', 19), ('c', 20), ('c', 21), ('c', 22), ('c', 23), ('c', 24), ('c', 25), ('c', 26), ('c', 27), ('c', 28), ('c', 